**DATASET 1: CREDIT CARD**

**ADABOOST Credit card**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from concurrent.futures import ThreadPoolExecutor
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier


data=pd.read_csv(r"C:\Users\NGUYEN PHUONG BICH\Downloads\creditcard.csv\creditcard.csv",sep=",")


X = data.drop(['class'], axis=1) #Lấy ra dữ liệu của các biến độc lập

y = (data['class']) #Lấy ra dữ liệu của biến phụ thuộc (nhãn)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Chia dữ liệu thành 2 phần train(80%) và test(20%)

#Biến đổi dữ liệu về phần phối chuẩn với giá trị trung bình = 0 và độ lệnh chuẩn = 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) #chuẩn hóa trên tập train
X_test = scaler.transform(X_test) #chuẩn hóa trên tập test


# Đào tạo bộ phân loại AdaBoost 
def train_adaboost(X_train, y_train, random_state):
    # n_estimators : số lượng cây cần huấn luyện
    # random_state : Đảm bảo mô hình con có tính ngẫu nhiên nhưng được tại lập với nếu cùng giá trị random_state
    # algorithm = SAMME : thuật toán tăng cường trọng số
    adaboost = AdaBoostClassifier(n_estimators=1, random_state=random_state, algorithm="SAMME") 
    adaboost.fit(X_train, y_train) # huấn luyện trên tập train
    return adaboost


n_estimators = 100 # Số lượng mô hình trong tập hợp

cores = [1,2,4,8]

for core in cores:

    # Thời gian bắt đầu đào tạo
    start_time = time.time()

    # Tạo nhóm luồng
    # max_workers: số luồng tối đa (tương ứng với số lõi CPU sử dụng)
    with ThreadPoolExecutor(max_workers=core) as executor:

        # Đào tạo nhiều mô hình song song
        #futures : danh sách việc huấn luyện mô hình
        # executor.submit() : gửi một tác vụ (huấn luyện mô hình con) vào hàng đợi thực thi 
        futures = [executor.submit(train_adaboost, X_train, y_train, i) for i in range(n_estimators)]

        # Lấy ra kết quả từ mỗi công việc (mô hình được huấn luyện)
        classifiers = [future.result() for future in futures]

    end_time = time.time() # Thời điểm kết thúc huấn luyện

    training_time = end_time - start_time # Thời gian huấn luyện mô hình

    print(f"Training Time: for {core} cores is {training_time:.2f} seconds")

    # Tổng hợp các dự đoán từ tất cả các mô hình
    def predict_ensemble(classifiers, X):

        predictions = np.zeros((X.shape[0], len(classifiers)), dtype=int) # Tạo ma trận dự đoán có X.shape[0] (số mẫu của X) hàng và 100 cột

        #enumerate(classifiers) : trả về từng cặp (index,value)
        # i : thứ tự của mô hình
        # classifier: mô hình thứ i
        for i, classifier in enumerate(classifiers):
            # Đưa dự đoán kết quả của mô hình thứ i cho mẫu X vào cột i của ma trận predictions
            predictions[:, i] = classifier.predict(X) 

        # Sử dụng phiếu bầu đa số cho dự đoán cuối cùng
        #.apply_along_axis() : tính toán theo hàng
        # .bincount() : đếm tần suất của mỗi nhãn
        # .argmax(): lấy nhãn xuất hiện nhiều nhất
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions)

    # Make predictions on the test data
    y_pred = predict_ensemble(classifiers, X_test)

    # Evaluate the model's performance
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for ADABoost: {accuracy * 100:.2f}%")

Training Time: for 1 cores is 106.41 seconds
Accuracy for ADABoost: 99.91%
Training Time: for 2 cores is 58.12 seconds
Accuracy for ADABoost: 99.91%
Training Time: for 4 cores is 32.21 seconds
Accuracy for ADABoost: 99.91%
Training Time: for 8 cores is 21.20 seconds
Accuracy for ADABoost: 99.91%


**RANDOM FOREST Credit Card**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from concurrent.futures import ThreadPoolExecutor
import time
import numpy as np
from sklearn.preprocessing import StandardScaler

data=pd.read_csv(r"C:\Users\NGUYEN PHUONG BICH\Downloads\creditcard.csv\creditcard.csv",sep=",")


X = data.drop(['class'], axis=1) #Lấy ra dữ liệu của các biến độc lập

y = (data['class']) #Lấy ra dữ liệu của biến phụ thuộc (nhãn)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Chia dữ liệu thành 2 phần train(80%) và test(20%)

#Biến đổi dữ liệu về phần phối chuẩn với giá trị trung bình = 0 và độ lệnh chuẩn = 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) #chuẩn hóa trên tập train
X_test = scaler.transform(X_test) #chuẩn hóa trên tập test


# Huấn luyện mô hình Random Forest
def train_tree(X_train, y_train, random_state):
    # n_estimators: số lượng cây cần huấn luyện
    # random_state : Đảm bảo mô hình có tính ngẫu nhiên nhưng được tại lập với nếu cùng giá trị random_state
    tree = RandomForestClassifier(n_estimators=1, random_state=random_state)
    tree.fit(X_train, y_train) #huấn luyện trên tập train
    return tree

# Số mô hình trong tập hợp (số mô hình học tổng hợp)
n_estimators = 100

cores = [1,2,4,8]

for core in cores:

    # Thời điểm bắt đầu huấn luyện
    start_time = time.time()

    #Tạo nhóm luồng
    # max_workers : số luồng tối đa (tương ứng với số lõi CPU sử dụng)
    with ThreadPoolExecutor(max_workers=core) as executor:
        # Huấn luyện các cây song song
        # executor.sunmit(): gửi tác , công việc (huấn luyện mô hình) vào hàng đợi thực thi
        futures = [executor.submit(train_tree, X_train, y_train, i) for i in range(n_estimators)] #danh sách các công việc

        # Lấy kết quả (cây đã được huấn luyện) từ mỗi công việc
        trees = [future.result() for future in futures]

    end_time = time.time() # Thời điểm kết thúc huấn luyện
    training_time = end_time - start_time #Thời gian huấn luyện mô hình
    
    print(f"Training Time Random Forest Credit: for {core} cores is {training_time:.2f} seconds")


    # Tổng hợp dự đoán từ tất cả các mô hình
    def predict_ensemble(trees, X):
        
        # Tạo ma trận dự đoán với X.shape[0] (số mẫu của X) hàng và 100 cột chỉ có số 0
        predictions = np.zeros((X.shape[0], len(trees)), dtype=int)
        
        for i, tree in enumerate(trees):
            predictions[:, i] = tree.predict(X) # Đưa kết quả dự đoán X của mô hình thứ i vào ma trận predictions
         
        # Sử dụng phiếu bầu đa số cho dự đoán cuối cùng
        #.apply_along_axis() : tính toán theo hàng
        # .bincount() : đếm tần suất của mỗi nhãn
        # .argmax(): lấy nhãn xuất hiện nhiều nhất
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions)



    # Đưa ra dự đoán về dữ liệu thử nghiệm
    y_pred = predict_ensemble(trees, X_test)


    # Đánh giá độ chính xác của mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy For Random Forest: {accuracy * 100:.2f}%")

Training Time Random Forest Credit: for 1 cores is 217.06 seconds
Accuracy For Random Forest: 99.96%
Training Time Random Forest Credit: for 2 cores is 113.42 seconds
Accuracy For Random Forest: 99.96%
Training Time Random Forest Credit: for 4 cores is 78.79 seconds
Accuracy For Random Forest: 99.96%
Training Time Random Forest Credit: for 8 cores is 46.45 seconds
Accuracy For Random Forest: 99.96%


*XGBOOST Credit Card*

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from concurrent.futures import ThreadPoolExecutor
import time
import numpy as np
from xgboost import XGBClassifier



data=pd.read_csv(r"C:\Users\NGUYEN PHUONG BICH\Downloads\creditcard.csv\creditcard.csv",sep=",")


X = data.drop(['class'], axis=1) #Lấy ra dữ liệu của các biến độc lập

y = (data['class']) #Lấy ra dữ liệu của biến phụ thuộc (nhãn)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Chia dữ liệu thành 2 phần train(80%) và test(20%)

#Biến đổi dữ liệu về phần phối chuẩn với giá trị trung bình = 0 và độ lệnh chuẩn = 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) #chuẩn hóa trên tập train
X_test = scaler.transform(X_test) #chuẩn hóa trên tập test


# Huấn luyện mô hình XGBoost
def train_xgboost_tree(X_train, y_train, random_state):
    # n_estimators: số cây cần huấn luyện
    # random_state : Đảm bảo mô hình có tính ngẫu nhiên nhưng được tại lập với nếu cùng giá trị random_state
    # use_label_encoder : tắt tính năng mã hóa nhãn thành số nguyên, vì mô hình mới không cần mã hóa
    tree = XGBClassifier(n_estimators=1, random_state=random_state, use_label_encoder=False)
    tree.fit(X_train, y_train)
    return tree

# Số mô hình trong tập hợp
n_estimators = 100

cores = [1,2,4,8]

for core in cores:
    
    # Thời điểm bắt đầu huấn luyện
    start_time = time.time()

    # Tạo nhóm luồng
    # max_workers: số luồng tối đa (tương ứng với số lõi CPU sử dụng)
    with ThreadPoolExecutor(max_workers=core) as executor:
        # Huấn luyện các cây song song
        # executor.submit(): thêm các tác vụ, công việc (huấn luyện mô hình) vào hàng đợi thực thi
        futures = [executor.submit(train_xgboost_tree, X_train, y_train, i) for i in range(n_estimators)] #danh sách các công việc

        # Lấy ra các cây đã được huấn luyện
        trees = [future.result() for future in futures]

    end_time = time.time() # thời điểm kết thúc huấn luyện
    training_time = end_time - start_time
    # print(f"Training Time: {training_time:.2f} seconds")
    print(f"Training Time XGBoost Credit: for {core} cores is {training_time:.2f} seconds")

    # Tổng hợp các dự đoán từ tất cả các cây
    def predict_ensemble(trees, X):
        #Tạo ma trận dự đoán với X.shape[0] (số mẫu của X) hàng và 100 cột với toàn số 0
        predictions = np.zeros((X.shape[0], len(trees)), dtype=int)
        for i, tree in enumerate(trees):
            predictions[:, i] = tree.predict(X) # Đưa dự đoán X của cây thứ i vào cột i của ma trận predictions
       
       # Sử dụng phiếu bầu đa số cho dự đoán cuối cùng
        #.apply_along_axis() : tính toán theo hàng
        # .bincount() : đếm tần suất của mỗi nhãn
        # .argmax(): lấy nhãn xuất hiện nhiều nhất
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions)

    #  Đưa ra dự đoán trên dữ liệu thử nghiệm
    y_pred = predict_ensemble(trees, X_test)

    # Đánh giá độ chính xác của mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for XGBoost Classifier: {accuracy * 100:.2f}%")

Training Time XGBoost Credit: for 1 cores is 21.82 seconds
Accuracy for XGBoost Classifier: 99.92%
Training Time XGBoost Credit: for 2 cores is 18.69 seconds
Accuracy for XGBoost Classifier: 99.92%
Training Time XGBoost Credit: for 4 cores is 17.34 seconds
Accuracy for XGBoost Classifier: 99.92%
Training Time XGBoost Credit: for 8 cores is 17.11 seconds
Accuracy for XGBoost Classifier: 99.92%


*KNN Credit Card*

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from concurrent.futures import ThreadPoolExecutor
import time
import numpy as np


from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

data=pd.read_csv(r"C:\Users\NGUYEN PHUONG BICH\Downloads\creditcard.csv\creditcard.csv",sep=",")


X = data.drop(['class'], axis=1) #Lấy ra dữ liệu của các biến độc lập

y = (data['class']) #Lấy ra dữ liệu của biến phụ thuộc (nhãn)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Chia dữ liệu thành 2 phần train(80%) và test(20%)

#Biến đổi dữ liệu về phần phối chuẩn với giá trị trung bình = 0 và độ lệnh chuẩn = 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) #chuẩn hóa trên tập train
X_test = scaler.transform(X_test) #chuẩn hóa trên tập test

# Huấn luyện mô hình kNN
def train_knn(X_train, y_train, random_state):
    knn = KNeighborsClassifier(n_neighbors=5)  # có thể thay đổi giá trị n_neighbors bằng bất kì giá trị nào mong muốn
    # công thức để tính khoảng cách có thể xác định thông qua đối số p, mặc đinh p=2 (sử dụng khoảng cách Euclid)
    knn.fit(X_train, y_train)
    return knn


# Số mô hình trong tập hợp
n_estimators = 100

cores = [1,2,4,8]

for core in cores:
   
    start_time = time.time()

    #Tạo nhóm luồng
    #max_workers : số luồng tối đa (tương ứng với số lõi CPU sử dụng)
    with ThreadPoolExecutor(max_workers=core) as executor:
        
        # Huấn luyện các mô hình song song
        # executor.submit(): gửi các tác vụ (huấn luyện mô hình) vào hàng đợi thực thi
        futures = [executor.submit(train_knn, X_train, y_train,i) for i in range(n_estimators)]

        # Lấy ra kết quả (mô hình đã được huấn luyện) của các tác vụ
        classifiers = [future.result() for future in futures]

    end_time = time.time()
    training_time = end_time - start_time
    
    print(f"Training Time KNN Credit: for {core} cores is {training_time:.2f} seconds")

    # Tổng hợp các dự đoán từ tất cả các mô hình
    def predict_ensemble(classifiers, X):

        #Tạo ma trận dự đoán với X.shape[0] (số mẫu của X) hàng và 100 cột với toàn số 0
        predictions = np.zeros((X.shape[0], len(classifiers)), dtype=int)
        for i, classifier in enumerate(classifiers):
            #Đưa dự đoán mẫu X của mô hình thứ i vào cột i của ma trận predictions
            predictions[:, i] = classifier.predict(X) 
        
        
        # Sử dụng phiếu bầu đa số cho dự đoán cuối cùng
        #.apply_along_axis() : tính toán theo hàng
        # .bincount() : đếm tần suất của mỗi nhãn
        # .argmax(): lấy nhãn xuất hiện nhiều nhất
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions)

    # Đưa ra dự đoán trên dữ liệu thử nghiệm
    y_pred = predict_ensemble(classifiers, X_test)

    # Đánh giá độ chính xác của mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")

Training Time KNN Credit: for 1 cores is 6.94 seconds
Training Time KNN Credit: for 2 cores is 7.29 seconds
Training Time KNN Credit: for 4 cores is 4.30 seconds
Training Time KNN Credit: for 8 cores is 5.01 seconds


_____________________________________________________

***********

___________________

**WINE QUALITY**

*ADABOOST Wine quality*

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from concurrent.futures import ThreadPoolExecutor
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier


data=pd.read_csv(r"C:\Users\NGUYEN PHUONG BICH\Downloads\winequality-white.csv",sep=";")


X = data.drop(['quality'], axis=1) #Lấy ra dữ liệu của các biến độc lập

y = (data['quality']) #Lấy ra dữ liệu của biến phụ thuộc (nhãn)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Chia dữ liệu thành 2 phần train(80%) và test(20%)

#Biến đổi dữ liệu về phần phối chuẩn với giá trị trung bình = 0 và độ lệnh chuẩn = 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) #chuẩn hóa trên tập train
X_test = scaler.transform(X_test) #chuẩn hóa trên tập test


# Đào tạo bộ phân loại AdaBoost 
def train_adaboost(X_train, y_train, random_state):
    # n_estimators : số lượng cây cần huấn luyện
    # random_state : Đảm bảo mô hình con có tính ngẫu nhiên nhưng được tại lập với nếu cùng giá trị random_state
    # algorithm = SAMME : thuật toán tăng cường trọng số
    adaboost = AdaBoostClassifier(n_estimators=1, random_state=random_state, algorithm="SAMME") 
    adaboost.fit(X_train, y_train) # huấn luyện trên tập train
    return adaboost


n_estimators = 100 # Số lượng mô hình trong tập hợp

cores = [1,2,4,8]

for core in cores:

    # Thời gian bắt đầu đào tạo
    start_time = time.time()

    # Tạo nhóm luồng
    # max_workers: số luồng tối đa (tương ứng với số lõi CPU sử dụng)
    with ThreadPoolExecutor(max_workers=core) as executor:

        # Đào tạo nhiều mô hình song song
        #futures : danh sách việc huấn luyện mô hình
        # executor.submit() : gửi một tác vụ (huấn luyện mô hình con) vào hàng đợi thực thi 
        futures = [executor.submit(train_adaboost, X_train, y_train, i) for i in range(n_estimators)]

        # Lấy ra kết quả từ mỗi công việc (mô hình được huấn luyện)
        classifiers = [future.result() for future in futures]

    end_time = time.time() # Thời điểm kết thúc huấn luyện

    training_time = end_time - start_time # Thời gian huấn luyện mô hình

    print(f"Training Time: for {core} cores is {training_time:.2f} seconds")

    # Tổng hợp các dự đoán từ tất cả các mô hình
    def predict_ensemble(classifiers, X):

        predictions = np.zeros((X.shape[0], len(classifiers)), dtype=int) # Tạo ma trận dự đoán có X.shape[0] (số mẫu của X) hàng và 100 cột

        #enumerate(classifiers) : trả về từng cặp (index,value)
        # i : thứ tự của mô hình
        # classifier: mô hình thứ i
        for i, classifier in enumerate(classifiers):
            # Đưa dự đoán kết quả của mô hình thứ i cho mẫu X vào cột i của ma trận predictions
            predictions[:, i] = classifier.predict(X) 

        # Sử dụng phiếu bầu đa số cho dự đoán cuối cùng
        #.apply_along_axis() : tính toán theo hàng
        # .bincount() : đếm tần suất của mỗi nhãn
        # .argmax(): lấy nhãn xuất hiện nhiều nhất
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions)

    # Make predictions on the test data
    y_pred = predict_ensemble(classifiers, X_test)

    # Evaluate the model's performance
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for ADABoost: {accuracy * 100:.2f}%")

Training Time: for 1 cores is 0.31 seconds
Accuracy for ADABoost: 44.08%
Training Time: for 2 cores is 0.18 seconds
Accuracy for ADABoost: 44.08%
Training Time: for 4 cores is 0.20 seconds
Accuracy for ADABoost: 44.08%
Training Time: for 8 cores is 0.17 seconds
Accuracy for ADABoost: 44.08%


*RANDOMFOREST Wine quality*

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from concurrent.futures import ThreadPoolExecutor
import time
import numpy as np
from sklearn.preprocessing import StandardScaler

data=pd.read_csv(r"C:\Users\NGUYEN PHUONG BICH\Downloads\winequality-white.csv",sep=";")


X = data.drop(['quality'], axis=1) #Lấy ra dữ liệu của các biến độc lập

y = (data['quality']) #Lấy ra dữ liệu của biến phụ thuộc (nhãn)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Chia dữ liệu thành 2 phần train(80%) và test(20%)

#Biến đổi dữ liệu về phần phối chuẩn với giá trị trung bình = 0 và độ lệnh chuẩn = 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) #chuẩn hóa trên tập train
X_test = scaler.transform(X_test) #chuẩn hóa trên tập test


# Huấn luyện mô hình Random Forest
def train_tree(X_train, y_train, random_state):
    # n_estimators: số lượng cây cần huấn luyện
    # random_state : Đảm bảo mô hình có tính ngẫu nhiên nhưng được tại lập với nếu cùng giá trị random_state
    tree = RandomForestClassifier(n_estimators=1, random_state=random_state)
    tree.fit(X_train, y_train) #huấn luyện trên tập train
    return tree

# Số mô hình trong tập hợp (số mô hình học tổng hợp)
n_estimators = 100

cores = [1,2,4,8]

for core in cores:

    # Thời điểm bắt đầu huấn luyện
    start_time = time.time()

    #Tạo nhóm luồng
    # max_workers : số luồng tối đa (tương ứng với số lõi CPU sử dụng)
    with ThreadPoolExecutor(max_workers=core) as executor:
        # Huấn luyện các cây song song
        # executor.sunmit(): gửi tác , công việc (huấn luyện mô hình) vào hàng đợi thực thi
        futures = [executor.submit(train_tree, X_train, y_train, i) for i in range(n_estimators)] #danh sách các công việc

        # Lấy kết quả (cây đã được huấn luyện) từ mỗi công việc
        trees = [future.result() for future in futures]

    end_time = time.time() # Thời điểm kết thúc huấn luyện
    training_time = end_time - start_time #Thời gian huấn luyện mô hình
    
    print(f"Training Time Random Forest Credit: for {core} cores is {training_time:.2f} seconds")


    # Tổng hợp dự đoán từ tất cả các mô hình
    def predict_ensemble(trees, X):
        
        # Tạo ma trận dự đoán với X.shape[0] (số mẫu của X) hàng và 100 cột chỉ có số 0
        predictions = np.zeros((X.shape[0], len(trees)), dtype=int)
        
        for i, tree in enumerate(trees):
            predictions[:, i] = tree.predict(X) # Đưa kết quả dự đoán X của mô hình thứ i vào ma trận predictions
         
        # Sử dụng phiếu bầu đa số cho dự đoán cuối cùng
        #.apply_along_axis() : tính toán theo hàng
        # .bincount() : đếm tần suất của mỗi nhãn
        # .argmax(): lấy nhãn xuất hiện nhiều nhất
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions)



    # Đưa ra dự đoán về dữ liệu thử nghiệm
    y_pred = predict_ensemble(trees, X_test)


    # Đánh giá độ chính xác của mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy For Random Forest: {accuracy * 100:.2f}%")

Training Time Random Forest Credit: for 1 cores is 0.66 seconds
Accuracy For Random Forest: 70.82%
Training Time Random Forest Credit: for 2 cores is 0.40 seconds
Accuracy For Random Forest: 70.82%
Training Time Random Forest Credit: for 4 cores is 0.28 seconds
Accuracy For Random Forest: 70.82%
Training Time Random Forest Credit: for 8 cores is 0.25 seconds
Accuracy For Random Forest: 70.82%


*XGBOOST Wine quality*

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from concurrent.futures import ThreadPoolExecutor
import time
import numpy as np
from xgboost import XGBClassifier


data=pd.read_csv(r"C:\Users\NGUYEN PHUONG BICH\Downloads\winequality-white.csv",sep=";")


X = data.drop(['quality'], axis=1) #Lấy ra dữ liệu của các biến độc lập

y = (data['quality']) #Lấy ra dữ liệu của biến phụ thuộc (nhãn)
label_encoder = LabelEncoder() #mã hóa nhãn
y = label_encoder.fit_transform(data['quality']) #chuyển đổi nhãn từ 3456789 thành 0123456

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Chia dữ liệu thành 2 phần train(80%) và test(20%)

#Biến đổi dữ liệu về phần phối chuẩn với giá trị trung bình = 0 và độ lệnh chuẩn = 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) #chuẩn hóa trên tập train
X_test = scaler.transform(X_test) #chuẩn hóa trên tập test


# Huấn luyện mô hình XGBoost
def train_xgboost_tree(X_train, y_train, random_state):
    # n_estimators: số cây cần huấn luyện
    # random_state : Đảm bảo mô hình có tính ngẫu nhiên nhưng được tại lập với nếu cùng giá trị random_state
    # use_label_encoder : tắt tính năng mã hóa nhãn thành số nguyên, vì mô hình mới không cần mã hóa
    tree = XGBClassifier(n_estimators=1, random_state=random_state, use_label_encoder=False)
    tree.fit(X_train, y_train)
    return tree

# Số mô hình trong tập hợp
n_estimators = 100

cores = [1,2,4,8]

for core in cores:
    
    # Thời điểm bắt đầu huấn luyện
    start_time = time.time()

    # Tạo nhóm luồng
    # max_workers: số luồng tối đa (tương ứng với số lõi CPU sử dụng)
    with ThreadPoolExecutor(max_workers=core) as executor:
        # Huấn luyện các cây song song
        # executor.submit(): thêm các tác vụ, công việc (huấn luyện mô hình) vào hàng đợi thực thi
        futures = [executor.submit(train_xgboost_tree, X_train, y_train, i) for i in range(n_estimators)] #danh sách các công việc

        # Lấy ra các cây đã được huấn luyện
        trees = [future.result() for future in futures]

    end_time = time.time() # thời điểm kết thúc huấn luyện
    training_time = end_time - start_time
    # print(f"Training Time: {training_time:.2f} seconds")
    print(f"Training Time XGBoost Credit: for {core} cores is {training_time:.2f} seconds")

    # Tổng hợp các dự đoán từ tất cả các cây
    def predict_ensemble(trees, X):
        #Tạo ma trận dự đoán với X.shape[0] (số mẫu của X) hàng và 100 cột với toàn số 0
        predictions = np.zeros((X.shape[0], len(trees)), dtype=int)
        for i, tree in enumerate(trees):
            predictions[:, i] = tree.predict(X) # Đưa dự đoán X của cây thứ i vào cột i của ma trận predictions
       
       # Sử dụng phiếu bầu đa số cho dự đoán cuối cùng
        #.apply_along_axis() : tính toán theo hàng
        # .bincount() : đếm tần suất của mỗi nhãn
        # .argmax(): lấy nhãn xuất hiện nhiều nhất
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions)

    #  Đưa ra dự đoán trên dữ liệu thử nghiệm
    y_pred = predict_ensemble(trees, X_test)

    # Đánh giá độ chính xác của mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for XGBoost Classifier: {accuracy * 100:.2f}%")

Training Time XGBoost Credit: for 1 cores is 0.84 seconds
Accuracy for XGBoost Classifier: 53.88%
Training Time XGBoost Credit: for 2 cores is 0.81 seconds
Accuracy for XGBoost Classifier: 53.88%
Training Time XGBoost Credit: for 4 cores is 0.59 seconds
Accuracy for XGBoost Classifier: 53.88%
Training Time XGBoost Credit: for 8 cores is 0.51 seconds
Accuracy for XGBoost Classifier: 53.88%


*KNN Wine quality*

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from concurrent.futures import ThreadPoolExecutor
import time
import numpy as np


from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

data=pd.read_csv(r"C:\Users\NGUYEN PHUONG BICH\Downloads\winequality-white.csv",sep=";")


X = data.drop(['quality'], axis=1) #Lấy ra dữ liệu của các biến độc lập

y = (data['quality']) #Lấy ra dữ liệu của biến phụ thuộc (nhãn)
label_encoder = LabelEncoder() #mã hóa nhãn
y = label_encoder.fit_transform(data['quality']) #chuyển đổi nhãn từ 3456789 thành 0123456

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Chia dữ liệu thành 2 phần train(80%) và test(20%)

#Biến đổi dữ liệu về phần phối chuẩn với giá trị trung bình = 0 và độ lệnh chuẩn = 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) #chuẩn hóa trên tập train
X_test = scaler.transform(X_test) #chuẩn hóa trên tập test


# Huấn luyện mô hình kNN
def train_knn(X_train, y_train, random_state):
    knn = KNeighborsClassifier(n_neighbors=5)  # có thể thay đổi giá trị n_neighbors bằng bất kì giá trị nào mong muốn
    # công thức để tính khoảng cách có thể xác định thông qua đối số p, mặc đinh p=2 (sử dụng khoảng cách Euclid)
    knn.fit(X_train, y_train)
    return knn


# Số mô hình trong tập hợp
n_estimators = 100

cores = [1,2,4,8]

for core in cores:
   
    start_time = time.time()

    #Tạo nhóm luồng
    #max_workers : số luồng tối đa (tương ứng với số lõi CPU sử dụng)
    with ThreadPoolExecutor(max_workers=core) as executor:
        
        # Huấn luyện các mô hình song song
        # executor.submit(): gửi các tác vụ (huấn luyện mô hình) vào hàng đợi thực thi
        futures = [executor.submit(train_knn, X_train, y_train,i) for i in range(n_estimators)]

        # Lấy ra kết quả (mô hình đã được huấn luyện) của các tác vụ
        classifiers = [future.result() for future in futures]

    end_time = time.time()
    training_time = end_time - start_time
    
    print(f"Training Time KNN Credit: for {core} cores is {training_time:.2f} seconds")

    # Tổng hợp các dự đoán từ tất cả các mô hình
    def predict_ensemble(classifiers, X):

        #Tạo ma trận dự đoán với X.shape[0] (số mẫu của X) hàng và 100 cột với toàn số 0
        predictions = np.zeros((X.shape[0], len(classifiers)), dtype=int)
        for i, classifier in enumerate(classifiers):
            #Đưa dự đoán mẫu X của mô hình thứ i vào cột i của ma trận predictions
            predictions[:, i] = classifier.predict(X) 
        
        
        # Sử dụng phiếu bầu đa số cho dự đoán cuối cùng
        #.apply_along_axis() : tính toán theo hàng
        # .bincount() : đếm tần suất của mỗi nhãn
        # .argmax(): lấy nhãn xuất hiện nhiều nhất
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions)

    # Đưa ra dự đoán trên dữ liệu thử nghiệm
    y_pred = predict_ensemble(classifiers, X_test)

    # Đánh giá độ chính xác của mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")

Training Time KNN Credit: for 1 cores is 0.41 seconds
Accuracy: 54.29%
Training Time KNN Credit: for 2 cores is 0.42 seconds
Accuracy: 54.29%
Training Time KNN Credit: for 4 cores is 0.39 seconds
Accuracy: 54.29%
Training Time KNN Credit: for 8 cores is 0.40 seconds
Accuracy: 54.29%
